# CYK

We will implement the CYK algorithm now. We will start by defining some useful utility classes. These include a `Rule` class that defines a rule from a PCFG and a `Tableau` class. The `Tableau` class stores the maximum probability of a parse with head tag `T` occuring in the subspan $[i,j]$

In [ ]:
class Rule:
  """ A single rule in PCFG of the form LHS ->  RHS with probability prob"""
  def __init__(self, lhs, rhs, prob):
    self.lhs = lhs
    self.rhs = rhs
    self.prob = prob

  def __eq__(self, other):
    # Used in the assertions below
    return self.lhs == other.lhs and self.rhs == other.rhs and self.prob == other.prob

rules = [Rule("S",["NP","VP"],0.80),
  Rule("S",["Aux","S'"],0.20),
  Rule("S'",["NP","VP"],1.00),
  Rule("NP",["Lufthansa"],0.45),
  Rule("NP",["you"],0.50),
  Rule("NP",["flights"],0.04),
  Rule("NP",["PN","Nom"],0.01),
  Rule("VP",["V","NP"],0.90),
  Rule("VP",["V","VP'"],0.10),
  Rule("VP'",["NP","NP"],1.00),
  Rule("PN",["Lufthansa"],1.0),
  Rule("V",["book"],1.00),
  Rule("Aux",["can"],1.00),
  Rule("Nom",["flights"],0.95),
  Rule("Nom",["PN","Nom"],0.05)]

class CYKTableau:
  """The CYK tableau records the likelihood of a parse with a head tag between 
  a span [i,j]"""
  def __init__(self, n):
    self.tab = [[{} for _ in range(n)] for _ in range(n)]

  def get_prob(self, i, j, tag):
    """Get the probability of the span [i,j] being tagged as tag"""
    return self.tab[i][j-1].get(tag, 0)

  def get_tags(self, i, j):
    """Get all tags with non-zero probability at span [i,j]"""
    return self.tab[i][j-1].keys()

  def update_prob(self, i, j, tag, p):
    """Set the probability at span [i,j] for tag to p, if it is large than the 
    current value"""
    if p > self.get_prob(i, j, tag):
      self.tab[i][j-1][tag] = p

  def __repr__(self):
    """Nice string version of the tableau (don't worry if you don't understand it!)"""
    s = ""
    for j in range(0,len(self.tab)):
      s += " " * 28 * j
      s += " | ".join("% 25s" % ",".join("%s: %.5f" % (k,v) for k,v in self.tab[i][i+j].items()) for i in range(0, len(self.tab)-j))
      s += "\n"
    return s

You should write the two functions bellow to lookup the matching preterminal and terminal rules from the grammar

In [ ]:
def find_preterminal_rule(word):
  """Find all preterminal rules for a given word"""
  return []

assert find_preterminal_rule("you") == [Rule("NP",["you"],0.50)]

def find_rules(a, b):
  """Find all rules that are like X -> A B"""
  return []

assert find_rules("NP","VP") == [Rule("S",["NP","VP"],0.80),
                                 Rule("S'",["NP","VP"],1.00)]

Now we will implement CYK, you should complete the marked lines to provide the appropriate probabilities. The output should look like the table we developed in the lecture.

In [ ]:
def cyk_parse(text):
  # make a tableau
  tableau = CYKTableau(len(text))
  # spans of length 1 to len(text) (inclusive)
  for span in range(1,len(text) + 1):
    # i is the index of the start of the span
    for i in range(len(text) - span + 1):
      # k is the index of the end of the span
      k = i + span
      if span == 1:
        for rule in find_preterminal_rule(text[i]):
          ### What to do here? ###
          pass 
      else:
        # j is the split in the rule
        for j in range(i + 1, k):
          # search for matching rules
          for tag1 in tableau.get_tags(i, j):
            for tag2 in tableau.get_tags(j, k):
              for rule in find_rules(tag1, tag2):
                ### What to do here? ###
                p = 0.0
  return tableau

cyk_parse(["can","you","book","Lufthansa","flights"])

             Aux: 1.00000 |               NP: 0.50000 |                V: 1.00000 |   NP: 0.45000,PN: 1.00000 |  NP: 0.04000,Nom: 0.95000
                                                      |                           |               VP: 0.40500 | VP': 0.01800,NP: 0.00950,Nom: 0.04750
                                                                                  |    S: 0.16200,S': 0.20250 |               VP: 0.00855
                                                                                                   S: 0.04050 |    S: 0.00342,S': 0.00428
                                                                                                                               S: 0.00086

Find another sentence that can be parsed with this grammar. Run the CYK algorithm on this sentence. What happens if we run this algorithm on an ungrammatical sentence?

In [ ]:
cyk_parse(["you","can","book","Lufthansa","flights"])

              NP: 0.50000 |              Aux: 1.00000 |                V: 1.00000 |   NP: 0.45000,PN: 1.00000 |  NP: 0.04000,Nom: 0.95000
                                                      |                           |               VP: 0.40500 | VP': 0.01800,NP: 0.00950,Nom: 0.04750
                                                                                  |                           |               VP: 0.00855
                                                                                                              |                          
                                                                                                                                         